# Ouput Parsers
* Format the answer of the LLM.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

## Parsing Outputs
* See the corresponding LangChain Documentation page [here](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/).
* Language models output text. But sometimes we would like to have those answers in a different format, like a JSON dictionary or a XML document. In order to achieve that, we use the Output Parsers.

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_openai import OpenAI

llmModel = OpenAI()

prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)

json_parser = SimpleJsonOutputParser()

json_chain = prompt | llmModel | json_parser

#### The previous prompt template includes the parser instructions

In [3]:
json_parser.get_format_instructions()

'Return a JSON object.'

In [4]:
json_chain.invoke({"question": "Tell me a joke"})

{'answer': 'What do you call a fake noodle? An Impasta.'}

#### Optionally, you can use Pydantic to define a custom output format

In [5]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

d:\Archived\Session one\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
# Define a Pydantic Object with the desired output format.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [7]:
# Define the parser referring the Pydantic Object
parser = JsonOutputParser(pydantic_object=Joke)

print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"setup": {"title": "Setup", "description": "question to set up a joke", "type": "string"}, "punchline": {"title": "Punchline", "description": "answer to resolve the joke", "type": "string"}}, "required": ["setup", "punchline"]}
```


In [8]:
# Define the parser referring the Pydantic Object
parser = JsonOutputParser(pydantic_object=Joke)

# Add the parser format instructions in the prompt definition.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


ChatModel = ChatOpenAI(model="gpt-4o-mini")
# Create a chain with the prompt and the parser
chain = prompt | ChatModel | parser

chain.invoke({"query": "قولي نكته عن ستالين"})

{'setup': 'لماذا كان ستالين دائمًا يحمل خريطة معه؟',
 'punchline': 'لأنه كان يحب أن يعرف أين يذهب لتوقيع البيعة!'}

# Quicker revision

In [9]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [10]:
load_dotenv()

True

In [15]:
def with_no_chain(input):
    prompt = ChatPromptTemplate.from_template(
        """Tell a story about {player} in 100 words"""
    )

    prompt = prompt.invoke(input)
    
    ChatModel = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))
    
    response = ChatModel.invoke(prompt)

    OutputParser = StrOutputParser()
    
    response_parsed = OutputParser.invoke(response)
    
    print(f"response parsed:\n{response_parsed}\n\n")

In [16]:
def using_chain(input):
    prompt = ChatPromptTemplate.from_template(
        """Tell a story about {player} in 100 words"""
    )

    ChatModel = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

    chain = prompt | ChatModel | StrOutputParser()
    
    response = chain.invoke(input)
    
    print(f"response Parsede:\n{response}")

In [17]:
with_no_chain("Ronaldo")

response parsed:
Cristiano Ronaldo stood at the edge of the pitch, the stadium pulsating with energy. As the final minutes ticked away, his team trailed by a goal. With fierce determination, he sprinted forward, evading defenders like shadows. The ball found his feet, and with a swift movement, he unleashed a powerful shot. Time slowed as the crowd held its breath. The ball soared into the net, igniting a wave of jubilation. Ronaldo’s teammates surrounded him, celebrating a last-minute equalizer. In that moment, he wasn't just a player; he was a symbol of hope, resilience, and pure passion for the game.




In [18]:
using_chain("Ronaldo")

response Parsede:
In a bustling square in Lisbon, a young boy named Miguel dreamed of becoming a football star like his idol, Cristiano Ronaldo. Every day after school, he practiced tirelessly, mimicking Ronaldo's signature moves. One sunny afternoon, Miguel spotted Ronaldo himself passing by. Gathering his courage, he approached and shared his dream. To his astonishment, Ronaldo smiled and invited him to train with him for an hour. They dribbled, shot, and laughed together. Miguel left inspired, not just by the skills learned but also by the kindness shown. That day sparked a lifelong passion, shaping Miguel into a star in his own right.


#### Note that using a chain made the code smaller, and more readable